In [1]:
# descriptive_set_tool.py
# Symbolic descriptive set theory tool: AST, syntactic classification, canonicalization,
# and heuristic "Borel isomorphism" checks.
#
# Limitations:
#  - Syntactic reasoning only (not semantic equality over R).
#  - "Countable" union/intersection represented as finite lists for computation.
#  - Projection handled conservatively (projection of Borel -> analytic).
#  - Heuristic isomorphism checks are incomplete.

from dataclasses import dataclass, field
from typing import List, Union, Optional, Tuple, Any
import functools
import itertools
import pprint

# --- Symbolic AST for set expressions ---

class SetExpr:
    """Base class for symbolic set expressions."""
    def canonical(self) -> "SetExpr":
        """Return a canonical (normalized) form of the expression for comparison heuristics."""
        return self

    def syntactic_class(self) -> Tuple[str, int, Optional[int]]:
        """
        Compute a conservative syntactic classification.
        Returns tuple (kind, level, proj_level)
          - kind in {"Borel-Sigma", "Borel-Pi", "Analytic", "Coanalytic", "Projective-Sigma", "Projective-Pi", "Unknown"}
          - level: for Borel, n (1 => Sigma^0_1 / Pi^0_1); for projective, n for Sigma^1_n/Pi^1_n
          - proj_level: None or number of projection-nesting depth (heuristic)
        """
        raise NotImplementedError

    def pretty(self) -> str:
        return self.__repr__()

@dataclass(frozen=True)
class AtomicOpen(SetExpr):
    name: str  # symbolic name of a basic open set
    def __repr__(self):
        return f"O({self.name})"
    def syntactic_class(self):
        # Basic open set: Sigma^0_1
        return ("Borel-Sigma", 1, 0)
    def canonical(self):
        return self

@dataclass(frozen=True)
class AtomicClosed(SetExpr):
    name: str
    def __repr__(self):
        return f"C({self.name})"
    def syntactic_class(self):
        # Basic closed set: Pi^0_1
        return ("Borel-Pi", 1, 0)
    def canonical(self):
        return self

@dataclass(frozen=True)
class Complement(SetExpr):
    child: SetExpr
    def __repr__(self):
        return f"¬({self.child})"
    def canonical(self):
        c = self.child.canonical()
        # double negation elimination heuristic
        if isinstance(c, Complement):
            return c.child.canonical()
        return Complement(c)
    def syntactic_class(self):
        kind, level, proj = self.child.syntactic_class()
        # Flip Sigma/Pi for Borel levels
        if kind.startswith("Borel-Sigma"):
            return ("Borel-Pi", level, proj)
        if kind.startswith("Borel-Pi"):
            return ("Borel-Sigma", level, proj)
        if kind == "Analytic":
            return ("Coanalytic", 1, (proj or 0))
        if kind == "Coanalytic":
            return ("Analytic", 1, (proj or 0))
        # If already projective at higher levels, flip Sigma/Pi component if available
        if kind.startswith("Projective-Sigma"):
            return (kind.replace("Sigma","Pi"), level, proj)
        if kind.startswith("Projective-Pi"):
            return (kind.replace("Pi","Sigma"), level, proj)
        return ("Unknown", 0, proj)

@dataclass
class CountableUnion(SetExpr):
    parts: List[SetExpr] = field(default_factory=list)
    def __repr__(self):
        return "⋃(" + ",".join(repr(p) for p in self.parts) + ")"
    def canonical(self):
        # canonicalize parts, sort by repr to give deterministic ordering
        parts_c = [p.canonical() for p in self.parts]
        parts_sorted = sorted(parts_c, key=lambda x: repr(x))
        # flatten nested unions
        flat = []
        for p in parts_sorted:
            if isinstance(p, CountableUnion):
                flat.extend(p.parts)
            else:
                flat.append(p)
        return CountableUnion(flat)
    def syntactic_class(self):
        # Use descriptive set theoretic syntactic rules:
        # - union of Sigma^0_n sets remains Sigma^0_max(n_i)
        # - union of Pi^0_n sets gives Sigma^0_{n+1} (countable union of Pi^0_n)
        # - if any part is analytic/projective, result upgrades appropriately
        sigma_levels = []
        pi_levels = []
        proj_kinds = []  # store projective/analytic kinds if present
        for p in self.parts:
            kind, level, proj = p.syntactic_class()
            if kind == "Analytic":
                proj_kinds.append(("Analytic", 1))
            elif kind == "Coanalytic":
                proj_kinds.append(("Coanalytic", 1))
            elif kind.startswith("Projective-Sigma") or kind.startswith("Projective-Pi"):
                proj_kinds.append((kind, level))
            elif kind.startswith("Borel-Sigma"):
                sigma_levels.append(level)
            elif kind.startswith("Borel-Pi"):
                pi_levels.append(level)
            elif kind == "Unknown":
                return ("Unknown", 0, None)
        if proj_kinds:
            # countable union of analytic sets is analytic; union may raise analytic/projective level
            # conservative answer: analytic (Sigma^1_1) if any analytic present
            return ("Analytic", 1, 1)
        max_sigma = max(sigma_levels) if sigma_levels else 0
        max_pi = max(pi_levels) if pi_levels else 0
        # minimal Sigma level is max(max_sigma, max_pi+1, 1)
        sigma_level = max(1, max_sigma, (max_pi + 1) if max_pi>0 else 1)
        return ("Borel-Sigma", sigma_level, 0)

@dataclass
class CountableIntersection(SetExpr):
    parts: List[SetExpr] = field(default_factory=list)
    def __repr__(self):
        return "⋂(" + ",".join(repr(p) for p in self.parts) + ")"
    def canonical(self):
        parts_c = [p.canonical() for p in self.parts]
        parts_sorted = sorted(parts_c, key=lambda x: repr(x))
        flat = []
        for p in parts_sorted:
            if isinstance(p, CountableIntersection):
                flat.extend(p.parts)
            else:
                flat.append(p)
        return CountableIntersection(flat)
    def syntactic_class(self):
        # dual to union: intersection of Pi stays Pi max level; intersection of Sigma -> Pi_{n+1}
        sigma_levels = []
        pi_levels = []
        proj_kinds = []
        for p in self.parts:
            kind, level, proj = p.syntactic_class()
            if kind == "Analytic":
                proj_kinds.append(("Analytic", 1))
            elif kind == "Coanalytic":
                proj_kinds.append(("Coanalytic", 1))
            elif kind.startswith("Projective-Sigma") or kind.startswith("Projective-Pi"):
                proj_kinds.append((kind, level))
            elif kind.startswith("Borel-Sigma"):
                sigma_levels.append(level)
            elif kind.startswith("Borel-Pi"):
                pi_levels.append(level)
            elif kind == "Unknown":
                return ("Unknown", 0, None)
        if proj_kinds:
            return ("Coanalytic", 1, 1)
        max_sigma = max(sigma_levels) if sigma_levels else 0
        max_pi = max(pi_levels) if pi_levels else 0
        pi_level = max(1, max_pi, (max_sigma + 1) if max_sigma>0 else 1)
        return ("Borel-Pi", pi_level, 0)

@dataclass
class Projection(SetExpr):
    # projection from a product space; symbolically existentially quantifying out a coordinate
    var: str  # name of the projected-out variable (symbolic)
    child: SetExpr
    def __repr__(self):
        return f"∃{self.var}.{self.child}"
    def canonical(self):
        return Projection(self.var, self.child.canonical())
    def syntactic_class(self):
        # Projection of a Borel set -> analytic (Sigma^1_1)
        # More generally, projection increases projective level by 1 if child is co/projective
        kind, level, proj = self.child.syntactic_class()
        if kind.startswith("Borel"):
            return ("Analytic", 1, (proj or 1))
        if kind == "Analytic":
            # projection of analytic typically stays analytic, but nesting could produce higher levels.
            return ("Analytic", 1, (proj or 1))
        if kind == "Coanalytic":
            # projection of coanalytic is Sigma^1_2 in general; but conservative: Projective Sigma^1_2
            return ("Projective-Sigma", 2, (proj or 1))
        if kind.startswith("Projective-Sigma"):
            return ("Projective-Sigma", level+1 if level else 2, (proj or 1)+1)
        if kind.startswith("Projective-Pi"):
            return ("Projective-Sigma", level+1 if level else 2, (proj or 1)+1)
        return ("Unknown", 0, None)

# --- Utilities for canonical structural comparison and heuristic invariants ---

def canonicalize(expr: SetExpr) -> SetExpr:
    return expr.canonical()

def syntactic_borel_signature(expr: SetExpr) -> str:
    k, level, proj = expr.syntactic_class()
    if k.startswith("Borel-Sigma"):
        return f"Σ^0_{level}"
    if k.startswith("Borel-Pi"):
        return f"Π^0_{level}"
    if k == "Analytic":
        return "Σ^1_1 (analytic)"
    if k == "Coanalytic":
        return "Π^1_1 (coanalytic)"
    if k.startswith("Projective-Sigma"):
        return f"Σ^1_{level} (projective)"
    if k.startswith("Projective-Pi"):
        return f"Π^1_{level} (projective)"
    return "Unknown"

def structural_invariants(expr: SetExpr) -> dict:
    """
    Compute a set of conservative invariants for the expression:
     - Borel/projective signature
     - AST size
     - multiset of atomic types used
    """
    atoms = []
    def walk(e):
        atoms_local = []
        if isinstance(e, AtomicOpen):
            atoms_local.append(("open", e.name))
        elif isinstance(e, AtomicClosed):
            atoms_local.append(("closed", e.name))
        elif isinstance(e, Complement):
            atoms_local.extend(walk(e.child))
        elif isinstance(e, CountableUnion) or isinstance(e, CountableIntersection):
            for p in e.parts:
                atoms_local.extend(walk(p))
        elif isinstance(e, Projection):
            atoms_local.extend(walk(e.child))
        return atoms_local
    atoms = walk(expr)
    return {
        "signature": syntactic_borel_signature(expr),
        "ast_size": ast_size(expr),
        "atoms_multiset": sorted(atoms)
    }

def ast_size(expr: SetExpr) -> int:
    if isinstance(expr, AtomicOpen) or isinstance(expr, AtomicClosed):
        return 1
    if isinstance(expr, Complement):
        return 1 + ast_size(expr.child)
    if isinstance(expr, CountableUnion) or isinstance(expr, CountableIntersection):
        return 1 + sum(ast_size(p) for p in expr.parts)
    if isinstance(expr, Projection):
        return 1 + ast_size(expr.child)
    return 0

def syntactic_equivalent(expr1: SetExpr, expr2: SetExpr) -> bool:
    """Deterministic structural comparison of canonical forms."""
    return repr(expr1.canonical()) == repr(expr2.canonical())

def heuristic_borel_isomorphic(expr1: SetExpr, expr2: SetExpr) -> Tuple[bool, str]:
    """
    Heuristic check for 'same up to Borel isomorphism' - extremely limited.
    We check:
      - exact canonical structural equality -> True
      - same signature and similar atom pattern -> maybe True (heuristic)
    Returns (decision_boolean, explanation_str)
    """
    c1 = canonicalize(expr1)
    c2 = canonicalize(expr2)
    if repr(c1) == repr(c2):
        return True, "Structurally identical (canonical forms match)."
    inv1 = structural_invariants(expr1)
    inv2 = structural_invariants(expr2)
    if inv1["signature"] != inv2["signature"]:
        return False, f"Different descriptive signatures: {inv1['signature']} vs {inv2['signature']}."
    # if signatures match and atom multisets similar, say 'likely'
    if inv1["atoms_multiset"] == inv2["atoms_multiset"]:
        return True, "Signatures match and atomic multisets identical (heuristic positive)."
    # weaker heuristic: same signature and similar number of atoms and AST size
    if inv1["ast_size"] == inv2["ast_size"] and len(inv1["atoms_multiset"]) == len(inv2["atoms_multiset"]):
        return False, "Equal AST size and signature but different atomic components — inconclusive; not declared isomorphic."
    return False, "Unable to certify Borel isomorphism with heuristics."

# --- Demonstration / Examples ---

def demo():
    print("=== Demonstration of the Descriptive Set Theory symbolic tool ===\n")
    # Basic atoms
    U = AtomicOpen("U")       # open
    V = AtomicOpen("V")
    F = AtomicClosed("F")     # closed
    G = AtomicClosed("G")

    # Build some expressions
    expr1 = CountableUnion([U, Complement(F)])          # U ∪ ¬F
    expr2 = CountableIntersection([Complement(U), G])   # ¬U ∩ G
    expr3 = Projection("y", CountableUnion([F, Complement(V)]))  # ∃y.(F ∪ ¬V)
    expr4 = CountableUnion([CountableIntersection([Complement(U), G]), V])
    expr5 = Complement(Complement(U))  # should canonicalize to U

    examples = [("U (open)", U), ("F (closed)", F), ("expr1 = U ∪ ¬F", expr1),
                ("expr2 = ¬U ∩ G", expr2), ("expr3 = ∃y.(F ∪ ¬V)", expr3),
                ("expr4 = (¬U ∩ G) ∪ V", expr4), ("expr5 = ¬¬U (canonicalizes to U)", expr5)]

    for name, ex in examples:
        print(f"Expression: {name}\n  repr: {repr(ex)}")
        sig = syntactic_borel_signature(ex)
        inv = structural_invariants(ex)
        print(f"  syntactic signature: {sig}")
        print(f"  AST size: {inv['ast_size']}; atoms: {inv['atoms_multiset']}")
        print()

    # Classification examples
    print("Classification examples:")
    for ex in [expr1, expr2, expr3, expr4, expr5]:
        print(f"  {repr(ex)} -> {syntactic_borel_signature(ex)}")

    # Heuristic isomorphism checks
    print("\nHeuristic 'Borel isomorphism' checks:")
    a = CountableUnion([U, V])
    b = CountableUnion([V, U])  # same up to ordering
    same, reason = heuristic_borel_isomorphic(a, b)
    print(f" a = {a}, b = {b} -> isomorphic? {same}. Reason: {reason}")

    c = CountableUnion([U, Complement(F)]) # earlier expr1
    d = CountableUnion([U, Complement(AtomicClosed("H"))]) # similar shape but different atom name
    same2, reason2 = heuristic_borel_isomorphic(c, d)
    print(f" c = {c}, d = {d} -> isomorphic? {same2}. Reason: {reason2}")

    # Some more interesting syntactic constructions
    s1 = CountableIntersection([AtomicOpen("A1"), AtomicOpen("A2")])  # intersection of opens -> Pi^0_2 (syntactically)
    s2 = CountableUnion([CountableIntersection([AtomicClosed("C1")]), AtomicOpen("B")])  # union of Pi and Sigma -> Sigma^0_2
    print()
    print(f" s1 = {s1} -> {syntactic_borel_signature(s1)}")
    print(f" s2 = {s2} -> {syntactic_borel_signature(s2)}")

    print("\nNOTE: This tool provides *syntactic* classification only. Many deep questions (actual equality, "
          "Borel isomorphism) are undecidable in general; heuristics are provided.\n")

if __name__ == "__main__":
    demo()


=== Demonstration of the Descriptive Set Theory symbolic tool ===

Expression: U (open)
  repr: O(U)
  syntactic signature: Σ^0_1
  AST size: 1; atoms: [('open', 'U')]

Expression: F (closed)
  repr: C(F)
  syntactic signature: Π^0_1
  AST size: 1; atoms: [('closed', 'F')]

Expression: expr1 = U ∪ ¬F
  repr: ⋃(O(U),¬(C(F)))
  syntactic signature: Σ^0_1
  AST size: 4; atoms: [('closed', 'F'), ('open', 'U')]

Expression: expr2 = ¬U ∩ G
  repr: ⋂(¬(O(U)),C(G))
  syntactic signature: Π^0_1
  AST size: 4; atoms: [('closed', 'G'), ('open', 'U')]

Expression: expr3 = ∃y.(F ∪ ¬V)
  repr: ∃y.⋃(C(F),¬(O(V)))
  syntactic signature: Σ^1_1 (analytic)
  AST size: 5; atoms: [('closed', 'F'), ('open', 'V')]

Expression: expr4 = (¬U ∩ G) ∪ V
  repr: ⋃(⋂(¬(O(U)),C(G)),O(V))
  syntactic signature: Σ^0_2
  AST size: 6; atoms: [('closed', 'G'), ('open', 'U'), ('open', 'V')]

Expression: expr5 = ¬¬U (canonicalizes to U)
  repr: ¬(¬(O(U)))
  syntactic signature: Σ^0_1
  AST size: 3; atoms: [('open', 'U')]

C